In [4]:
# программа расчета нейросети, реализующей функцию XOR
import numpy as np

# Обучающая выборка
X = np.array([ [1,0,0],[1,0,1],[1,1,0],[1,1,1] ])
y = np.array([[0,1,1,0]]).T

print(X)
print(y)

[[1 0 0]
 [1 0 1]
 [1 1 0]
 [1 1 1]]
[[0]
 [1]
 [1]
 [0]]


In [3]:
# -------------- Моделирование классификатора XOR -------------
# логистическая функция активации
def sigmoidfun(z):
    return 1 / (1 + np.exp(-z))


def sigmderiv(z):
    return sigmoidfun(z) * (1 - sigmoidfun(z))

def sigmoideriv(y):
    return y * (1 - y)

np.random.seed(1)
s0 = 2 * np.random.random((2, 3)) - 1
s1 = 2 * np.random.random((1, 3)) - 1

def xor_nnet(x, syn0=s0, syn1=s1):
    a1 = x.T
    a2 = sigmoidfun(np.dot(syn0, a1))
    if len(a2.shape) > 1:
        ones = np.ones(shape=(1, a2.shape[1]))
        a20 = np.vstack((ones, a2))
    else:
        a20 = np.array([1, a2[0], a2[1]])
    a3 = sigmoidfun(np.dot(syn1, a20))
    return a1, a20, a3

In [4]:
# -------------- Моделирование классификатора XOR -------------
# Задаем веса для классификатора, реализующего функцию XOR
syn0 = np.array([ [-5,-10,10],[-5,10,-10] ])
syn1 = np.array([ [-5,10,10] ])

ynet = np.array([[0.0, 0.0, 0.0, 0.0]]).T
err = ynet
for i in range(len(X)):
    x = X[i]
    a1,a2,ynet[i] = xor_nnet(x, syn0, syn1)
    err[i] = y[i] - ynet[i]
    print("Ответ классификатора для", x, "=", ynet[i])

Ответ классификатора для [1 0 0] = [-0.00764414]
Ответ классификатора для [1 0 1] = [0.00715279]
Ответ классификатора для [1 1 0] = [0.00715279]
Ответ классификатора для [1 1 1] = [-0.00764414]


In [5]:
# --------------- анализ ошибок классификации -----------------
# СРЕДНЕКВАДРАТИЧЕСКАЯ ОШИБКА (MSE)
print('MSE=', sum(err*err)/len(err))
# print('MSE=', np.mean(err*err))

# кроссэнтропия распределений (log-loss)
def plogp(p):
    if abs(p) < 1e-12:
        return 0
    return p*np.log2(p)

MSE= [5.47975957e-05]


In [7]:
# ------- попробуем обучить нейросеть простым град-м спуском ---------
# Попробуем обучить нейросеть XOR с помощью оптимизации
# случайно инициализируем веса, в среднем - 0
np.random.seed(3)
syn0 = 2*np.random.random((2,3)) - 1
syn1 = 2*np.random.random((1,3)) - 1

a1,a2,ynet = xor_nnet(X, syn0, syn1)
print(ynet)
# learning coef
alpha = 0.01

# Попробуем обучить нейросеть XOR
for j in range(600000):
    # проходим вперёд по слоям 0, 1 и 2
    a10,a20,a3 = xor_nnet(X, syn0, syn1)

    # как сильно мы ошиблись относительно нужной величины?
    a3_error = y - a3.T

    # рапространим ошибку на выходе 3-го слоя на вход слоя
    z3_delta = a3_error * sigmoideriv(a3.T)

    # распространим ошибки z3 на ошибки выходов a2 (входы для a3)
    a20_error = np.dot(syn1.T, z3_delta.T)
    # уберем 0-ю строку, содержащую добавленный +1
    a2 = a20[1:]
    a2_error = a20_error[1:]

    # рапространим ошибку на выходе 2-го слоя на вход слоя
    z2_delta = a2_error * sigmoideriv(a2)

    # рассчитаем дельта весов каждого слоя,
    # используя ошибки на входе (l+1) слоя и выходы l-го слоя
    syn1_delta = np.dot(z3_delta.T, a20.T)
    syn0_delta = np.dot(z2_delta, a10.T)

    syn1 += alpha* syn1_delta
    syn0 += alpha* syn0_delta

    if (j % 30000) == 0:
        print("Error:" + str(np.mean(np.abs(a3_error))))

print(syn1)
print(syn0)
a1,a2,ynet = xor_nnet(X, syn0, syn1)
print(ynet)

[[0.18096335 0.16557691 0.14980345 0.14153966]]
Error:0.5017806632374381
Error:0.4958867934391441
Error:0.3446072879383419
Error:0.09127435453125382
Error:0.05664042134229113
Error:0.043883382134332585
Error:0.036893905686122316
Error:0.03235945458778854
Error:0.029125933032138458
Error:0.026676011464766485
Error:0.024739776387334697
Error:0.023161204339646607
Error:0.021843162483658703
Error:0.020721678807102403
Error:0.01975271525426613
Error:0.01890485393499014
Error:0.018155010580695505
Error:0.017485799722054992
Error:0.01688384952268858
Error:0.016338687103310334
[[ -4.48638541 -10.51846703   9.74164621]]
[[-7.00981282  4.58522782  4.55025869]
 [-2.98183124  6.76632391  6.60203154]]
[[0.01753602 0.98482712 0.98498406 0.01564307]]


In [8]:
# ------- попробуем обучить нейросеть стохастическим градиентом ---------
# случайно инициализируем веса, в среднем - 0
np.random.seed(3)
syn0 = 2*np.random.random((2,3)) - 1
syn1 = 2*np.random.random((1,3)) - 1

# learning coef
alpha = 0.01
# pack size
pack_size = 3

# Попробуем обучить нейросеть XOR
for j in range(600000):
    # проходим вперёд по слоям 0, 1 и 2
    a10,a20,a3 = xor_nnet(X, syn0, syn1)
    # выбираем случайно pack_size объектов из обучающей выборки
    nobjs = np.random.choice(list(range(len(X))), pack_size, replace=False)
    # фильтруем нужные объекты и ответы
    a10 = a10[:, nobjs]
    a20 = a20[:, nobjs]
    a3 = a3[:, nobjs]
    yp = y[nobjs]

    # как сильно мы ошиблись относительно нужной величины?
    a3_error = yp - a3.T

    # рапространим ошибку на выходе 3-го слоя на вход слоя
    z3_delta = a3_error * sigmoideriv(a3.T)

    # распространим ошибки z3 на ошибки выходов a2 (входы для a3)
    a20_error = np.dot(syn1.T, z3_delta.T)
    # уберем 0-ю строку, содержащую добавленный +1
    a2 = a20[1:]
    a2_error = a20_error[1:]

    # рапространим ошибку на выходе 2-го слоя на вход слоя
    z2_delta = a2_error * sigmoideriv(a2)

    # рассчитаем дельта весов каждого слоя,
    # используя ошибки на входе (l+1) слоя и выходы l-го слоя
    syn1_delta = np.dot(z3_delta.T, a20.T)
    syn0_delta = np.dot(z2_delta, a10.T)

    syn1 += alpha* syn1_delta
    syn0 += alpha* syn0_delta

    if (j % 50000) == 0:
        print("Error:" + str(np.mean(np.abs(a3_error))))

print(syn1)
print(syn0)
a1,a2,ynet = xor_nnet(X, syn0, syn1)
print(a1)
print(a2)
print(ynet)

Error:0.6087197679120167
Error:0.4817883120080427
Error:0.17647421410589734
Error:0.061995343720827024
Error:0.04388113548099668
Error:0.035063979375143675
Error:0.030010002413792883
Error:0.027850712051608162
Error:0.024821579569469083
Error:0.022899323304130175
Error:0.020637568355052968
Error:0.020387557586109013
[[ -4.31594635 -10.19845644   9.41363496]]
[[-6.81597651  4.46157227  4.42166219]
 [-2.92654715  6.69726717  6.5037521 ]]
[[1 1 1 1]
 [0 0 1 1]
 [0 1 0 1]]
[[1.         1.         1.         1.        ]
 [0.00109492 0.08360729 0.08671634 0.88767986]
 [0.05085674 0.97280644 0.97748321 0.9999655 ]]
[[0.02086971 0.98181906 0.98203764 0.01878744]]


In [1]:
# ---------- реализация нейросети XOR с помощью KERAS
# import the necessary packages
from keras.optimizers import SGD, Adam, Nadam
from keras.utils import np_utils

# imports used to build the deep learning model
from keras.models import Sequential
from keras.layers.core import Activation
from keras.layers import Input, Dense
from keras.layers import BatchNormalization, Dropout, Lambda
from keras.models import Model

# from keras. import binary_crossentropy
from keras import backend as K

import matplotlib.pyplot as plt

def graph_training_history(history):
    plt.figure(1)

    # summarize history for accuracy

    plt.subplot(211)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')

    # summarize history for loss

    plt.subplot(212)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')

    plt.show()

In [2]:
def build_XORnnet(nx = 2):
    # Input
    a1 = Input(shape=(nx,))
    a2 = Dense(2, activation='sigmoid')(a1)
    a3 = Dense(1, activation='sigmoid')(a2)

    classificator = Model(a1, a3, name="XORnnet")
    # Return the constructed network architecture
    return classificator

In [6]:
model = build_XORnnet(nx = 2)
# Посмотрим на число параметров
model.summary()

Model: "XORnnet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 2)]               0         
                                                                 
 dense_4 (Dense)             (None, 2)                 6         
                                                                 
 dense_5 (Dense)             (None, 1)                 3         
                                                                 
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________


In [7]:
# Зададим основные параметры для обучения
batch_size = 4
Xb = X[:,1:]
yb = y

# Build and Compile the model
print("[INFO] Building and compiling the DNet model...")
# opt = SGD(lr=0.0002, momentum=0.0001, decay=0.00001, nesterov=True)
opt = Adam(lr=0.01)
# opt = Nadam(lr=0.0001)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

[INFO] Building and compiling the DNet model...


In [ ]:
# Check the argument whether to train the model
print("[INFO] Training the model...")
history = model.fit(Xb, yb,
                    batch_size= batch_size,
                    epochs=500,
                    shuffle= True,
                    #validation_split= 0.3,
                    validation_data=(Xb, yb),
                    verbose=1)

# Visualize the training history
graph_training_history(history)

# Training of the model is now complete
#model.summary()

In [13]:
# Use the test data to evaluate the model
# check it on ctrldata - проверяем точность модели на контрольной выборке
#ctrlLabels_softmax = np_utils.to_categorical(ctrlLabels, n_classes)
print("[INFO] Evaluating the model...")
# (loss, accuracy) = model.evaluate(ctrlData, ctrlLabels_softmax, verbose=1, batch_size= batch_size)
(loss, accuracy) = model.evaluate(Xb, yb, verbose=1, batch_size= batch_size)
print("[INFO] CONTROL accuracy: {:.2f}%".format(accuracy * 100))

[INFO] Evaluating the model...
1/1 [==============================] - 0s 33ms/step - loss: 0.1356 - accuracy: 1.0000
[INFO] CONTROL accuracy: 100.00%
